# TS-Plasmids Evaluation

In [1]:
%load_ext autoreload
%autoreload 2

from evaluation_scripts.strainplate import CreateStrainplate_layout_384
from evaluation_scripts.base import get_pathes, FileNames, get_t_end, multiply_setup_plates
from evaluation_scripts.instructions import Import_Raw_Instructions, get_turnover_id, infection_to_well_id, get_transfer_id, transfer_from_to_id
from evaluation_scripts.well_ident import Label_Replicates, build_dictionaries, makeID
import pandas as pd
import json
from pandas.io.json import json_normalize 

## Load basic information

In [2]:
jupyter_path, basepath, path, analysis_path, os = get_pathes("20220412_output")

In [3]:
additional_infos = os.path.join(path, "Notes", "info.json")
infos = {"strainplate_layout":"new", "setup":"new"}
if os.path.exists(additional_infos):
    with open(additional_infos) as json_data:
        i = json.load(json_data)
    infos.update(i)
print(infos)

{'strainplate_layout': 'new', 'setup': 'new'}


In [4]:
## Load all names of ... files in folder ... in the experiment folder ... and drop name of lock files (appear when opened)
instruction_file_names = FileNames(".csv", path, "platefiles")

## Create Dictionaries with basic infos
hue_infos, strategies, plate_rows, agar_strain_dict = build_dictionaries()

In [5]:
if infos["setup"] == "new":
    #First copy t0_P1_i to t0_P2-6_i_setup_copy. This is the easiest way to get the setupplate data into every strategy
    multiply_setup_plates(path, os)

## Create basic strainplate + exceptions

In [6]:
if infos["strainplate_layout"] == "new":
    t_end = get_t_end(instruction_file_names)
    strainplate = CreateStrainplate_layout_384(range(0,t_end+1), plate_rows)
    strainplate = Label_Replicates(strainplate, plate_rows)
    try:
        os.mkdir(analysis_path+os.sep+"obj")
    except:
        pass
else:
    from old_eval.ProperEvalFunctions import(CreateStrainplates_old_layout_384, CheckStrainPlates)
    t_end = get_t_end(instruction_file_names)
    strainplate = CreateStrainplates_old_layout_384(range(0,t_end+1), plate_rows)
    strainplate = CheckStrainPlates(path, strainplate)
    strainplate = Label_Replicates(strainplate, plate_rows)
    
    
obj_path = os.path.join(analysis_path, "obj")

if os.path.exists(os.path.join(analysis_path, "obj")) == False:
    os.mkdir(obj_path)
    
strainplate.to_pickle(obj_path+os.sep+"Strainplate.pkl")

## Write r-well infections/transfer/turnover to specific well ID
- Infection to
    - Notation not ideal ideal
        - The Instruction for t1, RWell 5 is to infect RWell 6
        - This means that RWell 6 is infected by Rwell 5 at t0 not t1...
    - Thats why the id instruction is written in the t0 well... 

In [7]:
## Just loads instructions into dataframe and sets correct index
print("Import_instructions")
Data = Import_Raw_Instructions(path,instruction_file_names)
# The transfer_to_well is actually wrong. It should actually be the transfered to r_well column... Otherwise they collide with turnover
Data = Data.rename(columns={"infection_to_well":"infection_to_rwell_with_time_error" })

## Adds which replicate is in which well
print("Label Replicates")
Data = Label_Replicates(Data, plate_rows)

## Get Turnover well IDs
print("Get turnover well id")
Data["turnover_id"] = Data.apply(lambda x:  get_turnover_id(strainplate, x) , axis=1)
Data = Data.sort_values(by=['transfer_n', "plate", 'rwell']).drop(columns=["transfer_to_well"])

Import_instructions
Label Replicates
Get turnover well id


In [8]:
Data[Data.transfer_n > 0]

,plate,well,row,col,tip,rep,rwell,transfer_n,turnover_strain,infection_to_rwell_with_time_error,treatment_with,turnover_id
ID,,,,,,,,,,,,
t1_P1_A1,1,1,A,1,1,0,1,1,NaN,NaN,none,None
t1_P1_A2,1,2,A,2,1,2,1,1,NaN,NaN,none,None
t1_P1_B1,1,25,B,1,1,1,1,1,NaN,NaN,none,None
t1_P1_B2,1,26,B,2,1,3,1,1,NaN,NaN,none,None
t1_P1_A3,1,3,A,3,1,0,2,1,NaN,NaN,none,None
...,...,...,...,...,...,...,...,...,...,...,...,...
t14_P6_P22,6,382,P,22,8,3,95,14,B_r,32.0,B,t14_S_H14
t14_P6_O23,6,359,O,23,8,0,96,14,B_r,NaN,B,t14_S_G13
t14_P6_O24,6,360,O,24,8,2,96,14,B_r,NaN,B,t14_S_G14


In [9]:
## Correct infections_for_time
tmp = Data.loc[Data.infection_to_rwell_with_time_error.isnull()==False, ].reset_index().copy()
idx = tmp.apply(lambda x: makeID(x.transfer_n-1, x.plate, x.row, x.col), axis=1)
Data.loc[idx.values, "infection_to_well"] = tmp["infection_to_rwell_with_time_error"].values


In [10]:
## Calculate the Well IDs based on the r_wells, replicates, time and plates for transfer and infection
# Thats why it will be dropped
print("Get infection well id (to well)")
#Data = r_well_to_id_infection(Data)
Data["infection_to_well_id"] = Data.apply(lambda x: infection_to_well_id(x, Data), axis = 1)

print("Get transfer well id")
# Calculate where the well transfers to..
Data["transfer_from_well_id"] = Data.apply(lambda x: get_transfer_id(x), axis=1)
Data = Data.drop(columns=["infection_to_rwell_with_time_error"])

Data.apply(lambda x: transfer_from_to_id(x, Data), axis=1).head(50)

Get infection well id (to well)
Get transfer well id


ID
t0_P1_A1     None
t0_P1_A2     None
t0_P1_B1     None
t0_P1_B2     None
t0_P1_A3     None
t0_P1_A4     None
t0_P1_B3     None
t0_P1_B4     None
t0_P1_A5     None
t0_P1_A6     None
t0_P1_B5     None
t0_P1_B6     None
t0_P1_A7     None
t0_P1_A8     None
t0_P1_B7     None
t0_P1_B8     None
t0_P1_A9     None
t0_P1_A10    None
t0_P1_B9     None
t0_P1_B10    None
t0_P1_A11    None
t0_P1_A12    None
t0_P1_B11    None
t0_P1_B12    None
t0_P1_A13    None
t0_P1_A14    None
t0_P1_B13    None
t0_P1_B14    None
t0_P1_A15    None
t0_P1_A16    None
t0_P1_B15    None
t0_P1_B16    None
t0_P1_A17    None
t0_P1_A18    None
t0_P1_B17    None
t0_P1_B18    None
t0_P1_A19    None
t0_P1_A20    None
t0_P1_B19    None
t0_P1_B20    None
t0_P1_A21    None
t0_P1_A22    None
t0_P1_B21    None
t0_P1_B22    None
t0_P1_A23    None
t0_P1_A24    None
t0_P1_B23    None
t0_P1_B24    None
t0_P1_C1     None
t0_P1_C2     None
dtype: object

In [11]:
# save the data
Data.to_pickle(analysis_path+os.sep+"obj"+os.sep+"01a_data_instructions.pkl")
Data.head()

,plate,well,row,col,tip,rep,rwell,transfer_n,turnover_strain,treatment_with,turnover_id,infection_to_well,infection_to_well_id,transfer_from_well_id,transfer_to_well_id
ID,,,,,,,,,,,,,,,
t0_P1_A1,1,1,A,1,1,0,1,0,A_r,none,t0_S_G9,NaN,NaN,None,t1_P1_A1
t0_P1_A2,1,2,A,2,1,2,1,0,A_r,none,t0_S_G10,NaN,NaN,None,t1_P1_A2
t0_P1_B1,1,25,B,1,1,1,1,0,A_r,none,t0_S_H9,NaN,NaN,None,t1_P1_B1
t0_P1_B2,1,26,B,2,1,3,1,0,A_r,none,t0_S_H10,NaN,NaN,None,t1_P1_B2
t0_P1_A3,1,3,A,3,1,0,2,0,B_r,none,t0_S_G13,NaN,NaN,None,t1_P1_A3
